In [2]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [3]:
SEASONS = list(range(2016, 2023))

In [4]:
DATA_DIR = "data"
STANDING_DIR = os.path.join(DATA_DIR, "standing")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [5]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries):
        time.sleep(sleep * 1)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print (await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [19]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDING_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

    html = await get_html(url, "#all_schedule")
    with open (save_path, "w+") as f:
        f.write(html)

In [20]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com


In [21]:
standings_files = os.listdir(STANDING_DIR)

In [27]:
standings_file = os.path.join(STANDING_DIR, standings_files[0])
with open(standings_file, 'r') as f:
    html = f.read()

soup = BeautifulSoup(html)
links = soup.find_all("a")
hrefs = [l.get("href") for l in links]
box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

In [28]:
box_scores

['https://www.basketball-reference.com/boxscores/202009010TOR.html',
 'https://www.basketball-reference.com/boxscores/202009010DEN.html',
 'https://www.basketball-reference.com/boxscores/202009020MIL.html',
 'https://www.basketball-reference.com/boxscores/202009020HOU.html',
 'https://www.basketball-reference.com/boxscores/202009030BOS.html',
 'https://www.basketball-reference.com/boxscores/202009030LAC.html',
 'https://www.basketball-reference.com/boxscores/202009040MIA.html',
 'https://www.basketball-reference.com/boxscores/202009040LAL.html',
 'https://www.basketball-reference.com/boxscores/202009050BOS.html',
 'https://www.basketball-reference.com/boxscores/202009050LAC.html',
 'https://www.basketball-reference.com/boxscores/202009060MIA.html',
 'https://www.basketball-reference.com/boxscores/202009060LAL.html',
 'https://www.basketball-reference.com/boxscores/202009070TOR.html',
 'https://www.basketball-reference.com/boxscores/202009070DEN.html',
 'https://www.basketball-reference